In this exercise, we want to find urls that matches given regular expression.

D = the content containing lots of urls (maybe big data)
RE = given set of regular expressions

In [140]:
# import libraries
import re
from difflib import SequenceMatcher

In [141]:
def MyProgressiveContinouseLength(X, Y):
    total_similarity = 0
    max_similarity = 0
    pre_similarity_count = 0
    for start_on in range(len(X)):
        similarity_count = 0
        idx_Y = 0
        str_ret = ""
        for i in range(start_on, len(X)):
            for j in range(idx_Y, len(Y)):
                if (X[i] == Y[j]):
                    similarity_count = similarity_count + 1
                    idx_Y = j + 1
                    str_ret = str_ret + str(X[i])
        #print(similarity_count)
        if similarity_count > pre_similarity_count and pre_similarity_count > 0:
            max_similarity = max(max_similarity, similarity_count)
            total_similarity = total_similarity + max_similarity
        pre_similarity_count = similarity_count
    return total_similarity

In [142]:
# test
X = "AGGTAB"
Y = "GXTXAYB"
print( "Length of LCS is ", MyProgressiveContinouseLength(X , Y) )

Length of LCS is  4


In [146]:
def RetError(str1, str2):
    #a = MyProgressiveContinouseLength(str1, str2)
    #return a / max(len(str1),len(str2))
    s = SequenceMatcher(None, str1, str2)
    return s.ratio()
    

In [147]:
def IsMatch(D, RE, startIdx, endIdx): 
    #print(RE)
    p = re.compile(RE)
    for i in range(startIdx, endIdx):
        #print(D[i])
        x = p.search(D[i])
        if x:
            print(x)
        else:
            print(RetError(D[i], RE))
    return x 

In [148]:
RE = ['https://MyWeb.com/(.*)']

D = ['https://MyWeb.com/1',
     'https://MyWeb.com/2',
     'https://MyWeb.com/3',
     'https://YourWeb.com/1',
     'https://YourWeb.com/2',
     'https://YourWeb.com/3',
    ]

X_arr = IsMatch(D, RE[0], 0, len(D))

p = re.compile(RE[0])
m = p.search(D[0])
print(m)

<_sre.SRE_Match object; span=(0, 19), match='https://MyWeb.com/1'>
<_sre.SRE_Match object; span=(0, 19), match='https://MyWeb.com/2'>
<_sre.SRE_Match object; span=(0, 19), match='https://MyWeb.com/3'>
0.7441860465116279
0.7441860465116279
0.7441860465116279
<_sre.SRE_Match object; span=(0, 19), match='https://MyWeb.com/1'>
